# FARM Building Blocks

Welcome to the FARM building blocks tutorial! There are many different ways to make use of this repository, but in this notebook, we will be going through the most import building blocks that will help you harvest the rewards of a successfully trained NLP model.

Happy FARMing!

## 1) Text Classification

GermEval 2018 (GermEval2018) (https://projects.fzai.h-da.de/iggsa/) is an open data set containing texts that need to be classified by whether they are offensive or not. There are a set of coarse and fine labels, but here we will only be looking at the coarse set which labels each example as either OFFENSE or OTHER. To tackle this task, we are going to build a classifier that is composed of Google's BERT language model and a feed forward neural network prediction head.

### Setup

In [4]:
# Install FARM
!pip install torch==1.8.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install farm==0.7.1

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 204kB 11.7MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 14.2MB 204kB/s 
     |████████████████████████████████| 1.5MB 36.1MB/s 
     |████████████████████████████████| 1.2MB 34.4MB/s 
     |████████████████████████████████| 327kB 41.2MB/s 
     |████████████████████████████████| 133kB 43.6MB/s 
     |████████████████████████████████| 2.5MB 30.5MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 348kB 29.9MB/s 
     |████████████████████████████████| 1.1MB 30.1MB/s 
     |████████████████████████████████| 153kB 42.5MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 81kB 1.3MB/s 
     |████████████████████████████████| 174kB 43.8MB/s 
     |████████████████████████████████| 2.9MB 32.7MB/s 
     |████████████████████████████████| 9

In [5]:
# Here are the imports we need

import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger


05/28/2021 12:11:01 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [6]:
# Farm allows simple logging of many parameters & metrics. Let's use the MLflow framework to track our experiment ...
# You will see your results on https://public-mlflow.deepset.ai/
ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab")



 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 


In [7]:
# We need to fetch the right device to drive the growth of our model
# Make sure that you have gpu turned on in this notebook by going to
# Runtime>Change runtime type and select GPU as Hardware accelerator.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cuda


### Data Handling

In [8]:
# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

tokenizer = Tokenizer.load(
    pretrained_model_name_or_path="bert-base-german-cased",
    do_lower_case=False)

05/28/2021 12:11:13 - INFO - filelock -   Lock 140431948383824 acquired on /root/.cache/huggingface/transformers/98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e.lock


05/28/2021 12:11:14 - INFO - filelock -   Lock 140431948383824 released on /root/.cache/huggingface/transformers/98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e.lock
05/28/2021 12:11:14 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'
05/28/2021 12:11:14 - INFO - filelock -   Lock 140431942803408 acquired on /root/.cache/huggingface/transformers/da299cdd121a3d71e1626f2908dda0d02658f42e925a3d6abd8273ec08cf41a6.31ccc255fc2bad3578089a3997f16b286498ba78c0adc43b5bb2a3f9a0d2c85c.lock


05/28/2021 12:11:14 - INFO - filelock -   Lock 140431942803408 released on /root/.cache/huggingface/transformers/da299cdd121a3d71e1626f2908dda0d02658f42e925a3d6abd8273ec08cf41a6.31ccc255fc2bad3578089a3997f16b286498ba78c0adc43b5bb2a3f9a0d2c85c.lock


In [9]:
from google.colab import files
uploaded = files.upload()

Saving GermEval21_Toxic_Train.csv to GermEval21_Toxic_Train (1).csv


In [22]:
# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor

LABEL_LIST = ["0", "1"]
metric = "f1_macro"
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir="/Users/xueyandong/Desktop/code/Data",
                                        label_list=LABEL_LIST,
                                        metric=metric,
                                        label_column_name="Sub1_Toxic",
                                        train_filename='GermEval21_Toxic_Train.csv', 
                                        dev_filename=None, 
                                        test_filename=None,
                                        dev_split=0.1,)

In [23]:
# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

05/28/2021 12:26:19 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
05/28/2021 12:26:19 - INFO - farm.data_handler.data_silo -   LOADING TRAIN DATA
05/28/2021 12:26:19 - INFO - farm.data_handler.data_silo -   ==================
05/28/2021 12:26:19 - INFO - farm.data_handler.data_silo -   Loading train set from: /Users/xueyandong/Desktop/code/Data/GermEval21_Toxic_Train.csv 
05/28/2021 12:26:19 - INFO - farm.data_handler.utils -    Couldn't find /Users/xueyandong/Desktop/code/Data/GermEval21_Toxic_Train.csv locally. Trying to download ...
05/28/2021 12:26:19 - INFO - farm.data_handler.utils -   downloading and extracting file Data to dir /Users/xueyandong/Desktop/code
05/28/2021 12:26:19 - ERROR - farm.data_handler.utils -   Cannot download Data. Unknown data source.


FileNotFoundError: ignored

### Modeling

In FARM, we make a strong distinction between the language model and prediction head so that you can mix and match different building blocks for your needs.

For example, in the transfer learning paradigm, you might have the one language model that you will be using for both document classification and NER. Or perhaps you have a pretrained language model which you would like to adapt to your domain, then use it for a downstream task such as question answering. 

All this is possible within FARM and requires only the replacement of a few modular components, as we shall see below.

Let's first have a look at how we might set up a model.

In [ ]:
# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "bert-base-german-cased"

language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

05/20/2021 15:44:54 - INFO - farm.modeling.language_model -   
05/20/2021 15:44:54 - INFO - farm.modeling.language_model -   LOADING MODEL
05/20/2021 15:44:54 - INFO - farm.modeling.language_model -   =============
05/20/2021 15:44:54 - INFO - farm.modeling.language_model -   Could not find bert-base-german-cased locally.
05/20/2021 15:44:54 - INFO - farm.modeling.language_model -   Looking on Transformers Model Hub (in local cache and online)...
05/20/2021 15:44:56 - INFO - filelock -   Lock 140131129640080 acquired on /root/.cache/huggingface/transformers/5236eea09283e87ba7c16d0571a12520ed4f076869f3d943fdbfaaa34b71e419.953a553bf3928a893b8cacf8d8c46ce6c565c095f062120aa0773821285cde25.lock


05/20/2021 15:45:23 - INFO - filelock -   Lock 140131129640080 released on /root/.cache/huggingface/transformers/5236eea09283e87ba7c16d0571a12520ed4f076869f3d943fdbfaaa34b71e419.953a553bf3928a893b8cacf8d8c46ce6c565c095f062120aa0773821285cde25.lock


05/20/2021 15:45:26 - INFO - farm.modeling.language_model -   Automatically detected language from language model name: german
05/20/2021 15:45:26 - INFO - farm.modeling.language_model -   Loaded bert-base-german-cased


In [ ]:
# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

05/20/2021 15:45:28 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
05/20/2021 15:45:28 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.7548528 1.4809586]


In [ ]:
# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

### Training

In [ ]:
# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

LEARNING_RATE = 2e-5
N_EPOCHS = 1

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

05/20/2021 15:45:36 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 2e-05}'
05/20/2021 15:45:38 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
05/20/2021 15:45:38 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 14.100000000000001, 'num_training_steps': 141}'


In [ ]:
# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

In [ ]:
!nvidia-smi

Thu May 20 15:45:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    59W / 149W |    851MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model = trainer.train()

05/20/2021 15:46:00 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 16/16 [00:07<00:00,  2.26it/s]
05/20/2021 15:48:26 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

In [ ]:
# Test your model on a sample (Inference)
from farm.infer import Inferencer
from pprint import PrettyPrinter

infer_model = Inferencer(processor=processor, model=model, task_type="text_classification", gpu=True)

basic_texts = [
    {"text": "Martin ist ein Idiot"},
    {"text": "Martin Müller spielt Handball in Berlin"},
]
result = infer_model.inference_from_dicts(dicts=basic_texts)
PrettyPrinter().pprint(result)


05/20/2021 15:50:16 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
05/20/2021 15:50:16 - INFO - farm.utils -   Using device: CUDA 
05/20/2021 15:50:16 - INFO - farm.utils -   Number of GPUs: 1
05/20/2021 15:50:16 - INFO - farm.utils -   Distributed Training: False
05/20/2021 15:50:16 - INFO - farm.utils -   Automatic Mixed Precision: None
05/20/2021 15:50:16 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
05/20/2021 15:50:16 - INFO - farm.infer -    0    0 
05/20/2021 15:50:16 - INFO - farm.infer -   /w\  /w\
05/20/2021 15:50:16 - INFO - farm.infer -   /'\  / \
05/20/2021 15:50:16 - INFO - farm.infer -     
05/20/2021 15:50:16 - INFO - farm.data_handler.processor -   *** Show 1 random examples ***
05/20/2021 15:50:16 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ 

[{'predictions': [{'context': 'Martin ist ein Idiot',
                   'end': None,
                   'label': 'OFFENSE',
                   'probability': 0.85085064,
                   'start': None},
                  {'context': 'Martin Müller spielt Handball in Berlin',
                   'end': None,
                   'label': 'OTHER',
                   'probability': 0.76713055,
                   'start': None}],
  'task': 'text_classification'}]


# Switch to NER

In a transfer learning paradigm, there is a core computation that is shared amongst all tasks. FARM's modular structure means that you can easily swap out different building blocks to make the same language model work for many different tasks.

We can adapt the above text classification model to NER by simply switching out the processor and prediction head.

In [ ]:
# Import the new building blocks

from farm.data_handler.processor import NERProcessor
from farm.modeling.prediction_head import TokenClassificationHead
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab_NER")


In [ ]:
# This processor will preprocess the data for the CoNLL03 NER task
ner_labels = ["[PAD]", "X", "O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-OTH", "I-OTH"]

ner_processor = NERProcessor(tokenizer=tokenizer, 
                             max_seq_len=128, 
                             data_dir="../data/conll03-de",
                             label_list=ner_labels,
                             metric="seq_f1")

In [ ]:
# This prediction head is also a feed forward neural network but expects one
# vector per token in the input sequence and will generate a set of logits
# for each input

ner_prediction_head = TokenClassificationHead(num_labels=len(ner_labels))

05/20/2021 15:51:03 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 13]


In [ ]:
# We can integrate these new pieces with the rest using this code
# It is pretty much the same structure as what we had above for text classification

BATCH_SIZE = 32
EMBEDS_DROPOUT_PROB = 0.1
LEARNING_RATE = 2e-5
N_EPOCHS = 1
N_GPU = 1

data_silo = DataSilo(
    processor=ner_processor,
    batch_size=BATCH_SIZE)

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[ner_prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_token"],
    device=device)

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS,
    device=device)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device,
)

05/20/2021 15:51:05 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
05/20/2021 15:51:05 - INFO - farm.data_handler.data_silo -   LOADING TRAIN DATA
05/20/2021 15:51:05 - INFO - farm.data_handler.data_silo -   ==================
05/20/2021 15:51:05 - INFO - farm.data_handler.data_silo -   Loading train set from: ../data/conll03-de/train.txt 
05/20/2021 15:51:05 - ERROR - farm.data_handler.utils -   Separator 	 for dataset German CONLL03 does not match the requirements. Setting seperator to whitespace
05/20/2021 15:51:05 - INFO - farm.data_handler.utils -    Couldn't find ../data/conll03-de/train.txt locally. Trying to download ...
05/20/2021 15:51:05 - INFO - farm.data_handler.utils -   downloading and extracting file conll03-de to dir /data
05/20/2021 15:51:07 - INFO - farm.data_handler.data_silo -   Got ya 2 paral

In [ ]:
model = trainer.train()

05/20/2021 15:51:24 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 90/90 [00:46<00:00,  1.95it/s]
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-p